In [1]:
!pip3 install tune-sklearn "ray[tune]"
!pip3 install cloudpickle imbalanced-learn scikit-optimize

You should consider upgrading via the '/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/venvs/sustainbench/bin/python3 -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/venvs/sustainbench/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import sys
import os
import math
sys.path.append('/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction')

from utils.get_data_loader import SustainBenchTextDataset
from sklearn.linear_model import Ridge, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import time # Just to compare fit times
from sklearn.metrics import r2_score, roc_auc_score, classification_report

In [3]:
classification_cutoff_dict = {'asset_index': 0, 'sanitation_index': 3, 'water_index': 3, 'women_edu': 5}
TARGETS = ['asset_index', 'sanitation_index', 'water_index', 'women_edu']
FEATURE_TYPES = ['target_sentence', 'document', 'target_sentence_document']

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.exceptions import ConvergenceWarning
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC


def evaluate_hyperparameter_optimization_regression(X_train, y_train, X_test, y_test):
    clf = Ridge()
    
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()

    y_pred = clf.predict(X_test)

    print(f'baseline fit time: {end - start}, r^2 score: {round(r2_score(y_test, y_pred), 5)}')

    
    parameters = {
       'alpha': np.logspace(-2,1,20)
    }
    # n_jobs=-1 enables use of all cores like Tune does
    sklearn_search = GridSearchCV(
       Ridge(),
       parameters,
       n_jobs=-1
    )

    start = time.time()
    sklearn_search.fit(X_train, y_train)
    end = time.time()
    y_pred = sklearn_search.predict(X_test)
    
    print(f'sklearn GridSearchCV fit time: {end - start}, r^2 score: {round(r2_score(y_test, y_pred), 5)}')
    print()

    
def evaluate_hyperparameter_optimization_classification(X_train, y_train, X_test, y_test, use_smote=True):
    if use_smote:
        oversample = SMOTE()
        X_train, y_train = oversample.fit_resample(X_train, y_train)

#     clf = RandomForestClassifier()

#     start = time.time()
#     clf.fit(X_train, y_train)
#     end = time.time()

#     y_pred = clf.predict(X_test)

#     print(f'baseline fit time: {end - start}, roc auc: {round(roc_auc_score(y_test, y_pred), 5)}')
#     print(classification_report(y_test, y_pred))

#     pipe = Pipeline(steps=[("scaler", MaxAbsScaler()), ("logistic", LogisticRegression(max_iter=500))])
#     parameters = {
#         "logistic__C": np.logspace(0,2,10)
#     }

#     # n_jobs=-1 enables use of all cores like Tune does
#     sklearn_search = GridSearchCV(
#        pipe,
#        parameters,
#        n_jobs=-1
#     )

#     start = time.time()
#     sklearn_search.fit(X_train, y_train)
#     end = time.time()
#     y_pred = sklearn_search.predict(X_test)

#     print(f'sklearn GridSearchCV fit time: {end - start}')
#     print(classification_report(y_test, y_pred))

#     print()


    for model in ['svm', 'lr', 'rf']:
        print(model)
        if model == 'svm':
            base_estimator = Pipeline(steps=[("scaler", MaxAbsScaler()), ("svm", SVC(gamma='scale'))])
            param_grid = {'svm__kernel': ('linear', 'rbf'),'svm__C': np.logspace(-1,2,20)}
            sklearn_search = HalvingGridSearchCV(
                base_estimator, 
                param_grid, 
                cv=3,
                factor=2,
                max_resources=100,
                scoring='accuracy',
                error_score=0
            )
        elif model == 'lr':
            base_estimator = Pipeline(steps=[("scaler", MaxAbsScaler()), ("lr", LogisticRegression(max_iter=500))])
            param_grid = {'lr__C': np.logspace(-1,2,20)}
            sklearn_search = HalvingGridSearchCV(
                base_estimator, 
                param_grid, 
                cv=3,
                factor=2,
                max_resources=100,
                scoring='accuracy',
                error_score=0
            )
        else:
            base_estimator = RandomForestClassifier(random_state=0)
            param_grid = {'max_depth': [3, 5, 10], 'min_samples_split': [2, 5, 10]}
            sklearn_search = HalvingGridSearchCV(
                base_estimator, 
                param_grid, 
                cv=3,
                factor=2,
                resource='n_estimators',
                max_resources=100,
                scoring='accuracy',
                error_score=0
            )

        start = time.time()
        sklearn_search.fit(X_train, y_train)
        end = time.time()
        y_pred = sklearn_search.predict(X_test)

        print(f'sklearn HalvingGridSearchCV fit time: {end - start}, roc auc: {round(roc_auc_score(y_test, y_pred), 5)}')
        print(sklearn_search.best_estimator_)
        print(classification_report(y_test, y_pred))

        print()



In [5]:
for target in TARGETS:
    for feature_type in ['target_sentence', 'document', 'target_sentence_document']:
        ds = SustainBenchTextDataset(
            data_dir=f'/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/data/', 
            feature_type=feature_type, 
            target=target,
            model_type='classification',
            classification_cutoff=classification_cutoff_dict[target]
        )

        print(target, feature_type)
        X_train, y_train = ds.get_data('train')
        X_test, y_test = ds.get_data('test')
        print(X_train.shape, X_test.shape)

        evaluate_hyperparameter_optimization_classification(X_train, y_train, X_test, y_test) 

asset_index target_sentence
(1944, 384) (390, 384)
svm
sklearn HalvingGridSearchCV fit time: 0.9301249980926514, roc auc: 0.57035
Pipeline(steps=[('scaler', MaxAbsScaler()), ('svm', SVC(C=1.8329807108324356))])
              precision    recall  f1-score   support

           0       0.65      0.71      0.68       235
           1       0.50      0.43      0.46       155

    accuracy                           0.60       390
   macro avg       0.57      0.57      0.57       390
weighted avg       0.59      0.60      0.59       390


lr
sklearn HalvingGridSearchCV fit time: 0.6052558422088623, roc auc: 0.56205
Pipeline(steps=[('scaler', MaxAbsScaler()),
                ('lr',
                 LogisticRegression(C=0.14384498882876628, max_iter=500))])
              precision    recall  f1-score   support

           0       0.68      0.46      0.55       235
           1       0.45      0.66      0.54       155

    accuracy                           0.54       390
   macro avg       0.5

/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/venvs/sustainbench/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


sklearn HalvingGridSearchCV fit time: 1.4087810516357422, roc auc: 0.60404
Pipeline(steps=[('scaler', MaxAbsScaler()),
                ('lr', LogisticRegression(C=100.0, max_iter=500))])
              precision    recall  f1-score   support

           0       0.57      0.57      0.57       275
           1       0.64      0.63      0.63       322

    accuracy                           0.61       597
   macro avg       0.60      0.60      0.60       597
weighted avg       0.61      0.61      0.61       597


rf
sklearn HalvingGridSearchCV fit time: 37.772931814193726, roc auc: 0.53057
RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=96,
                       random_state=0)
              precision    recall  f1-score   support

           0       0.49      0.59      0.53       275
           1       0.57      0.47      0.52       322

    accuracy                           0.53       597
   macro avg       0.53      0.53      0.53       597
weighted avg       0.

sklearn HalvingGridSearchCV fit time: 901.7867257595062, roc auc: 0.54949
RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=96,
                       random_state=0)
              precision    recall  f1-score   support

           0       0.46      0.66      0.54      5498
           1       0.64      0.44      0.52      7641

    accuracy                           0.53     13139
   macro avg       0.55      0.55      0.53     13139
weighted avg       0.57      0.53      0.53     13139


women_edu target_sentence_document
(4136, 684) (1096, 684)
svm
sklearn HalvingGridSearchCV fit time: 5.931267023086548, roc auc: 0.58908
Pipeline(steps=[('scaler', MaxAbsScaler()),
                ('svm', SVC(C=0.14384498882876628, kernel='linear'))])
              precision    recall  f1-score   support

           0       0.54      0.67      0.60       506
           1       0.64      0.51      0.57       590

    accuracy                           0.58      1096
   macro avg  

/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/venvs/sustainbench/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


sklearn HalvingGridSearchCV fit time: 2.442631959915161, roc auc: 0.56364
Pipeline(steps=[('scaler', MaxAbsScaler()),
                ('lr', LogisticRegression(C=69.51927961775606, max_iter=500))])
              precision    recall  f1-score   support

           0       0.51      0.70      0.59       506
           1       0.62      0.42      0.51       590

    accuracy                           0.55      1096
   macro avg       0.57      0.56      0.55      1096
weighted avg       0.57      0.55      0.55      1096


rf
sklearn HalvingGridSearchCV fit time: 88.62702798843384, roc auc: 0.57253
RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=96,
                       random_state=0)
              precision    recall  f1-score   support

           0       0.51      0.76      0.61       506
           1       0.65      0.39      0.49       590

    accuracy                           0.56      1096
   macro avg       0.58      0.57      0.55      1096
weighted av

In [6]:
# for target in TARGETS:
#     for feature_type in FEATURE_TYPES:
#         ds = SustainBenchTextDataset(
#             data_dir=f'/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/data/', 
#             feature_type=feature_type, 
#             target=target,
#             model_type='regression',
#             classification_cutoff=classification_cutoff_dict[target]
#         )

#         print(target, feature_type)
#         X_train, y_train = ds.get_data('train')
#         X_test, y_test = ds.get_data('test')
#         print(X_train.shape, X_test.shape)
        
#         evaluate_hyperparameter_optimization_regression(X_train, y_train, X_test, y_test) 